<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/Rest_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import csv
import pandas as pd
import numpy  as np

In [5]:
#read file
file = open('combine.csv')
type(file)
csvreader = csv.reader(file)

In [ ]:
#check the header
header = []
header = next(csvreader)
header

In [28]:
# data cleaning 
data = pd.DataFrame(csvreader)
data.replace(' ', np.NaN)
pct_null = data.isnull().sum() / len(data)
missing_features = pct_null[pct_null > 0.80].index
data.drop(missing_features, axis=1, inplace=True)

In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('combine.csv')
post = df['Postal Code']
sub = df['Subdivision Name']
#for i in range(len(sub)):
    #post[i] = post[i] + ' ' + sub[i].split()[0]
    
#post

df.fillna(0)
pr = df['Close Price']

for i in range(len(pr)):
    if(pr[i] == 'Close Price'):
        print(i)
        df = df.drop([i])

prs = df['Close Price']
final = []
for item in prs:
    item = int(item)
    final.append(item)
df['Close Price'] = final
df_a = df.groupby(['Bathrooms Total Interger', 'Bedrooms Total'])['Close Price'].mean()
df_a

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,6,7,8,9,10,11,13,20,23,27,29,31,35,48,49,51,55,56,57,59,60,61,62,63,76,80,81,82,83,84,86,95,96,97,98,101,102,103,105,111,115,117,123,124,125,129,130,131,133,136,138,140,141,144,152,153,154,157,158,159,161,163,172,177,178,192,193,194,199,200,202,203,209,212,216,219,238,240,245,248,251) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


3789
6776
9525
13521
16851
19796
24415


Bathrooms Total Interger  Bedrooms Total
0.0                       0.0               233110.000000
                          3.0               225000.000000
                          4.0                 2350.000000
1.0                       0.0                 1198.000000
                          1.0                67279.428571
                                                ...      
7                         4                   7225.000000
                          5                 400000.000000
                          6                  20000.000000
8                         5                   8500.000000
                          8                   3500.000000
Name: Close Price, Length: 83, dtype: float64